In [4]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import folium
import requests
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

Matplotlib is building the font cache; this may take a moment.


In [2]:
# Load the dataset into a pandas dataframe
df = pd.read_csv('/african_conflicts.csv', encoding='ISO-8859-1')

C:\Users\Akwo\anaconda3\python.exe


In [3]:
# Print the shape of the dataframe
print("Shape of the dataframe: ", df.shape)

SyntaxError: unexpected character after line continuation character (1929608216.py, line 1)

In [ ]:
# Print the data types of the columns
print("Data types of the columns: ")
print(df.dtypes)

In [ ]:
# Print the first 5 rows of the dataframe
print("First 5 rows of the dataframe: ")
print(df.head())

In [ ]:
# Print summary statistics for the numeric columns
print("Summary statistics for numeric columns: ")
print(df.describe())

In [ ]:
# Check for missing values
print(df.isnull().sum())

In [ ]:
# Convert date columns to datetime with format 'dd/mm/yyyy'
df['EVENT_DATE'] = pd.to_datetime(df['EVENT_DATE'], format='%d/%m/%Y')

In [ ]:
# Check for missing values
print(df.isnull().sum())

In [ ]:
# Replace inconsistent values in the "GEO_PRECISION" column
df['GEO_PRECISION'] = df['GEO_PRECISION'].replace({-9: np.nan})

In [ ]:
text = " ".join(df['NOTES'].dropna()) # Concatenate all the text data from your dataset into a single string

In [ ]:
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = set(STOPWORDS),
                min_font_size = 10)

In [ ]:
wordcloud.generate(text)

In [ ]:
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

In [ ]:
# convert the 'EVENT_DATE' column to datetime format
df['EVENT_DATE'] = pd.to_datetime(df['EVENT_DATE'])

In [ ]:
# create a new column for the year
df['YEAR'] = df['EVENT_DATE'].dt.year

In [ ]:
# group the data by year and count the number of events in each year
attacks_by_year = df.groupby('YEAR').size()

In [ ]:
# create a line plot of the frequency of attacks over time
plt.plot(attacks_by_year.index, attacks_by_year.values)
plt.xlabel('Year')
plt.ylabel('Number of Attacks')
plt.title('Frequency of Attacks in Africa, 1997-2017')
plt.show()

In [ ]:
# Set figure size
plt.figure(figsize=(12,6))

# Create plot
ax = df.groupby(['YEAR'])['EVENT_ID_CNTY'].count().plot()

# Set x-axis label
ax.set_xlabel("Year")

# Set y-axis label
ax.set_ylabel("Number of Conflicts")

# Set title
ax.set_title("Frequency of Conflicts in Africa (1997-2017)")

# Show plot
plt.show()

In [ ]:
corr = df['FATALITIES'].corr(df['YEAR'])

print("Pearson's correlation coefficient: ", corr)

In [ ]:
plt.scatter(df['YEAR'], df['FATALITIES'])
plt.xlabel('Year')
plt.ylabel('Fatalities')
plt.show()

In [ ]:
# Filter the dataset to only include events in 1997 and 2001
events_97_01 = df[(df.YEAR == 1997) | (df.YEAR == 2001)]

# Display the notes column for the filtered dataset
print(events_97_01.NOTES)

In [ ]:
# Get the count of conflicts per country
country_counts = df['COUNTRY'].value_counts()

# Plot the top 10 countries with the highest number of conflicts
top_10_countries = country_counts.head(10)
plt.figure(figsize=(10,6))
plt.bar(top_10_countries.index, top_10_countries.values)
plt.title('Top 10 Countries with the Highest Number of Conflicts')
plt.xlabel('Country')
plt.ylabel('Number of Conflicts')
plt.show()

In [ ]:
# Convert the "EVENT_DATE" column to datetime format
df['EVENT_DATE'] = pd.to_datetime(df['EVENT_DATE'], format='%d/%m/%Y')

# Group the dataset by year and count the number of conflicts per year
conflict_count = df.groupby(df['EVENT_DATE'].dt.year)['EVENT_ID_CNTY'].count()

# Plot the trend of conflicts over the years
plt.figure(figsize=(10,6))
plt.plot(conflict_count.index, conflict_count.values)
plt.title('Trend of Conflicts over the Years')
plt.xlabel('Year')
plt.ylabel('Number of Conflicts')
plt.show()

In [ ]:
# Extract rows where either ACTOR1 or ACTOR2 contains the word "Somalia"
somalia_conflicts = df[(df['ACTOR1'].str.contains("Somalia")) | (df['ACTOR2'].str.contains("Somalia"))]

In [ ]:
# Filter the dataframe to only include Riots/Protests
riots_df = df[df['EVENT_TYPE'] == 'Riots/Protests'].copy()

# Group the data by year and count the number of events
riots_year_count = riots_df.groupby(riots_df['YEAR']).size().reset_index(name='counts')

# Plot the line graph
plt.plot(riots_year_count['YEAR'], riots_year_count['counts'])
plt.title('Frequency of Riots/Protests by Year')
plt.xlabel('Year')
plt.ylabel('Number of Riots/Protests')
plt.show()

In [ ]:
actor_df = df['ACTOR1'].value_counts().reset_index()
actor_df.columns = ['Actor', 'Frequency']
actor_df = actor_df.head(10)

In [ ]:
plt.barh(actor_df['Actor'], actor_df['Frequency'], color='cornflowerblue')
plt.xlabel('Frequency')
plt.ylabel('Actor')
plt.title('Top 10 Most Active Actors in African Conflicts')
plt.tight_layout()
plt.show()

In [ ]:
# Get the top 10 most active actors
top_actors = df['ACTOR1'].value_counts().head(10)

# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(top_actors.index, top_actors.values, color='purple')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Actor')
plt.ylabel('Number of Conflicts')
plt.title('Top 10 Most Active Actors in Conflicts')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fatalities_df = df[['YEAR', 'FATALITIES']].groupby('YEAR').sum().reset_index()

plt.plot(fatalities_df['YEAR'], fatalities_df['FATALITIES'])
plt.xlabel('Year')
plt.ylabel('Total Fatalities')
plt.title('Trends in Fatalities')
plt.show()

In [ ]:
Geographic distribution of conflicts: A heat map or a choropleth map can be created to show the number of conflicts or the intensity of conflicts in different countries or regions.

In [ ]:
conflicts_by_country = df.groupby('COUNTRY').size().reset_index(name='counts')

In [ ]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
response = requests.get(f'{url}/world-countries.json')
world_geojson = response.json()

m = folium.Map(location=[30, 0], zoom_start=2)
folium.GeoJson(
    world_geojson,
    name='World conflicts',
    tooltip=folium.features.GeoJsonTooltip(fields=['name'], labels=False),
    style_function=lambda x: {'fillColor': 'red', 'color': 'black', 'weight': 2, 'fillOpacity': 0.5}
).add_to(m)

m

In [ ]:
m = folium.Map(location=[0, 0], zoom_start=2)
folium.Choropleth(
    geo_data=world_geojson,
    name='choropleth',
    data=conflicts_by_country,
    columns=['COUNTRY', 'counts'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Conflicts',
).add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
Temporal distribution of conflicts: A line graph or a bar graph can be created to show the number of conflicts over time.

In [ ]:
# Convert the 'EVENT_DATE' column to a pandas datetime format
df['EVENT_DATE'] = pd.to_datetime(df['EVENT_DATE'], errors='coerce')

# Extract the year from the 'EVENT_DATE' column and create a new 'YEAR' column
df['YEAR'] = df['EVENT_DATE'].dt.year

# Group the conflicts by year and count the number of conflicts each year
conflicts_by_year = df.groupby('YEAR').size().reset_index(name='counts')

# Create a line graph of the number of conflicts by year
plt.plot(conflicts_by_year['YEAR'], conflicts_by_year['counts'])
plt.xlabel('Year')
plt.ylabel('Number of Conflicts')
plt.title('Temporal Distribution of Conflicts')
plt.show()

In [ ]:
# Calculate counts of each conflict type
conflict_counts = df['EVENT_TYPE'].value_counts()

# Create a stacked bar chart
conflict_counts.plot(kind='bar', stacked=True)

# Set plot labels and title
plt.xlabel('Conflict Type')
plt.ylabel('Number of Conflicts')
plt.title('Types of Conflicts')
plt.show()

In [ ]:
# Create a pivot table to get the number of fatalities by country and year
fatalities_by_country = df.pivot_table(index='COUNTRY', columns='YEAR', values='FATALITIES', aggfunc='sum')

# Get the top 20 countries with the most fatalities
top_countries = fatalities_by_country.sum(axis=1).nlargest(20)

# Filter the pivot table to only include top countries
fatalities_by_country = fatalities_by_country.loc[top_countries.index]

# Create a scatter plot to show the intensity of conflicts
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(fatalities_by_country.mean(axis=1),  # X-axis: average number of fatalities
           fatalities_by_country.std(axis=1),   # Y-axis: standard deviation of fatalities
                      s=fatalities_by_country.sum(axis=1) * 0.5,  # Size of bubble: total number of fatalities
           alpha=0.5)  # Transparency of bubbles
ax.set_xlabel('Average number of fatalities')
ax.set_ylabel('Standard deviation of fatalities')
ax.set_title('Intensity of conflicts by country')
plt.show()


In [ ]:
The above is a scatter plot with the average number of fatalities on the X-axis, the standard deviation of fatalities on the Y-axis, and the size of the bubble representing the total number of fatalities. The transparency of the bubbles is set to 0.5 to allow overlapping bubbles to be more visible. The top 20 countries with the most fatalities are included in the plot.